# url, date, title 데이터 가져오기

In [1]:
from selenium import webdriver
from bs4 import BeautifulSoup
from urllib.parse import quote
import requests
import time
import pandas as pd

In [2]:
def naver_crawling(result, naver_url):
    options = webdriver.ChromeOptions()
    options.add_argument("headless")
    options.add_argument("--start-maximized")

    driver = webdriver.Chrome("chromedriver.exe", chrome_options=options)
    driver.get(naver_url)

    prev_height = driver.execute_script("return document.body.scrollHeight")

    # 웹페이지 맨 아래까지 무한 스크롤
    while True:        
        driver.execute_script("window.scrollTo(0,document.body.scrollHeight)")
        time.sleep(2)
        curr_height = driver.execute_script("return document.body.scrollHeight")

        if(curr_height == prev_height):
            break
        else:
            prev_height = driver.execute_script("return document.body.scrollHeight")

    # url, date, title
    contents = driver.find_elements_by_css_selector('div.total_area')

    for content in contents:
        url = content.find_element_by_css_selector('a.api_txt_lines.total_tit').get_attribute('href')
        date = content.find_element_by_css_selector('span.sub_time.sub_txt').text
        title = content.find_element_by_css_selector('a.api_txt_lines.total_tit').text

        result.append([url, date, title])
        
    driver.quit()
        
    return result

In [3]:
# url, date, title 정보
result = []

# 20200201 ~ 20200930
keyword = "한달살기"
naver_url = "https://search.naver.com/search.naver?where=blog&query="+ quote(keyword) + "&sm=tab_opt&nso=so%3Ar%2Cp%3Afrom20200201to20200930"
result = naver_crawling(result, naver_url)

# 20201001 ~ 20210531
naver_url = "https://search.naver.com/search.naver?where=blog&query="+ quote(keyword) + "&sm=tab_opt&nso=so%3Ar%2Cp%3Afrom20201001to20210531"
result = naver_crawling(result, naver_url)

print(len(result))

<ipython-input-2-e3966ea796fe>:6: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome("chromedriver.exe", chrome_options=options)


2000


In [4]:
naver_df = pd.DataFrame(result)
naver_df.columns = ['url', 'date', 'title']

# hashtag, 글내용 데이터 가져오기

In [5]:
hashtags = []
contents = []
i = 0

options = webdriver.ChromeOptions()
options.add_argument("headless")
    
driver = webdriver.Chrome("chromedriver.exe", chrome_options=options)

for blog_url in naver_df['url']:
    hashtag_list = []
    content_list = []
    
    # 몇번째인지 확인하는 코드, 추후 삭제 예정
    i += 1
    if i % 20 == 0:
        print(str(i) + "번째")
    
    driver.get(blog_url)
    time.sleep(5)
    
    # 해시태그 크롤링
    try:
        driver.switch_to.frame("mainFrame")
        page = driver.page_source
        soup = BeautifulSoup(page, "html.parser")

    except:
        page = driver.page_source
        soup = BeautifulSoup(page, "html.parser")
    
    try:
        hashtag_data = soup.select("a.item.pcol2.itemTagfont._setTop")
    
        for hashtag in hashtag_data:
            hashtag_list.append(hashtag.text)
    
    except:
        hashtag_list.append("에러")

    hashtags.append(hashtag_list)
    
    # 내용 크롤링
    try:
        content_data = soup.select("div.se-component.se-text.se-l-default")

        for content in content_data:
            content_list.append(content.text)
        
        if len(content_list) == 0:
            content_data = soup.select("div.post-view p")
            
            for content in content_data:
                content_list.append(content.text)
        
        if len(content_list) == 0:
            content_data = soup.select("div.se_component_wrap.sect_dsc.__se_component_area p")
            
            for content in content_data:
                content_list.append(content.text)
                
        if len(content_list) == 0:
            content_data = soup.select("div.post-view div")

            for content in content_data:
                content_list.append(content.text)
        
    except:
        content_list.append("에러")
        
    contents.append(content_list)
    
    # 추후 삭제 예정
    if i % 500 == 0:
        print("에러방지용 sleep 진행중")
        time.sleep(180)
    
driver.quit()

<ipython-input-5-666ce259e167>:8: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome("chromedriver.exe", chrome_options=options)


20번째
40번째
60번째
80번째
100번째
120번째
140번째
160번째
180번째
200번째
220번째
240번째
260번째
280번째
300번째
320번째
340번째
360번째
380번째
400번째
420번째
440번째
460번째
480번째
500번째
에러방지용 sleep 진행중
520번째
540번째
560번째
580번째
600번째
620번째
640번째
660번째
680번째
700번째
720번째
740번째
760번째
780번째
800번째
820번째
840번째
860번째
880번째
900번째
920번째
940번째
960번째
980번째
1000번째
에러방지용 sleep 진행중
1020번째
1040번째
1060번째
1080번째
1100번째
1120번째
1140번째
1160번째
1180번째
1200번째
1220번째
1240번째
1260번째
1280번째
1300번째
1320번째
1340번째
1360번째
1380번째
1400번째
1420번째
1440번째
1460번째
1480번째
1500번째
에러방지용 sleep 진행중
1520번째
1540번째
1560번째
1580번째
1600번째
1620번째
1640번째
1660번째
1680번째
1700번째
1720번째
1740번째
1760번째
1780번째
1800번째
1820번째
1840번째
1860번째
1880번째
1900번째
1920번째
1940번째
1960번째
1980번째
2000번째
에러방지용 sleep 진행중


In [6]:
naver_df['hashtags'] = hashtags
naver_df['contents'] = contents

# 엑셀 파일로 저장하기

In [7]:
naver_df.to_excel(keyword + '.xlsx', index = False)

In [8]:
naver_df

,url,date,title,hashtags,contents
0,https://blog.naver.com/utiful_sun92?Redirect=L...,2020.09.15.,[제주 한달살기_기록] ep1.19박20일 제주살이 조천리야 반가워!,"[#씨앤하우스, #조천카페, #틈, #제주한달살기, #제주도여행, #제주도]","[\n\n\n\n​\n\n\n\n, \n\n\n\n​미래의 나 보라고 이렇게 잘 쉬..."
1,https://blog.naver.com/obh1680?Redirect=Log&lo...,2020.09.07.,오션뷰의 제주 한달살기 숙소,"[#제주한달살기숙소, #탄탈루스]","[\n\n\n\n​\n\n\n\n, \n\n\n\n​\n\n\n\n, \n\n\n\..."
2,https://blog.naver.com/misoagnes?Redirect=Log&...,2020.07.30.,"제주 한달 살기 차귀도 배낚시, 협재 해수욕장 모래놀이_4","[#제주도한달살기, #제주한달살기, #차귀도배낚시, #수용횟집배낚시, #수용배낚시,...",[\n\n\n\n​​​​​​제주도 한달살기 4일째!날씨가 좋지 않았지만 낚시를 하고...
3,https://blog.naver.com/mouche007?Redirect=Log&...,2020.02.13.,코타키나발루 숙소 한달살기 가성비 추천,"[#코타키나발루숙소, #코타키나발루한달살기, #코타스윗]",[\n\n\n\n​​​​사원 모두가 블루와 그린으로 이루어졌다는 블루모스크가 창밖으...
4,https://blog.naver.com/youriful?Redirect=Log&l...,2020.02.26.,발리한달살기 #21. 우붓 요가반 수업 후기,"[#발리한달살기, #발리우붓요가, #우붓요가반]",[\n\n\n\nDay 1620190628-0726 발리 한달살기 이야기 \n\n\...
...,...,...,...,...,...
1995,https://blog.naver.com/shtaak?Redirect=Log&log...,2021.04.24.,한달살기 전문숙소인 블루하우스 방역하기,"[#바닷가바로앞, #블루하우스, #동상일몽, #한달살기, #방역, #살충, #사회적...","[최근 바닷가바로앞 블루하우스인 동상일몽으로 , 한달살기 하러 오시는 분들 중에 ,..."
1996,https://blog.naver.com/jeju_lotte?Redirect=Log...,2021.05.04.,"제주도 호텔, 제주도 한 달 살기 숙소 추천! 롯데호텔 제주, 제주...","[#제주도, #제주, #롯데호텔제주, #롯데호텔, #국내여행지추천, #국내여행, #...","[\n\n\n\n 일상생활 중 혹은제주 여행을 하면서 문득, ‘제주에 살아보고 싶다..."
1997,https://blog.naver.com/j_yun93?Redirect=Log&lo...,2020.12.27.,"윤쿠잉 제주 한달살기 : 14일차, 서쪽 여행 끝, 협재해수욕장...","[#윤쿠잉제주라이프, #제주도한달살기]","[\n\n\n\n윤쿠잉 제주 한달살기 : 14일차, 서쪽 여행 끝, 협재해수욕장, ..."
1998,https://blog.naver.com/manim159?Redirect=Log&l...,2021.02.10.,(남해한달살기)(남해보름살기)(남해일주일살기),"[#남해날마다소풍, #남해한달살기, #남해보름살기, #남해일주일살기, #경남한달살기...","[#남해한달살기 #남해보름살기 #남해일주일살기, ​, 프라이빗 바다가 펼처진 아..."
